In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler



In [3]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_samples, h, w = lfw_people.images.shape
X = lfw_people.data
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [5]:

scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_train_stacked = np.column_stack((X_train_normalized))
X_train_stacked.shape #N^2 * M

(1850, 966)

In [6]:
cov_matrix = np.cov(X_train_stacked) #cov= A*A^T 
print(cov_matrix)

[[ 1.00103628e+00  9.33115905e-01  7.78384382e-01 ...  3.05566064e-03
  -1.26172584e-02 -2.63080423e-02]
 [ 9.33115905e-01  1.00103627e+00  8.98681257e-01 ...  1.39497783e-02
   2.47042400e-03 -1.70691009e-02]
 [ 7.78384382e-01  8.98681257e-01  1.00103625e+00 ...  7.85702075e-03
  -7.09239945e-04 -2.43257895e-02]
 ...
 [ 3.05566064e-03  1.39497783e-02  7.85702075e-03 ...  1.00103628e+00
   8.98058875e-01  7.15818695e-01]
 [-1.26172584e-02  2.47042400e-03 -7.09239945e-04 ...  8.98058875e-01
   1.00103629e+00  9.10392628e-01]
 [-2.63080423e-02 -1.70691009e-02 -2.43257895e-02 ...  7.15818695e-01
   9.10392628e-01  1.00103629e+00]]


In [7]:
cov_matrix.shape
#### N^2 * N^2


(1850, 1850)

In [8]:
# now to find k significant eigenfaces (dimensionality reduction) k<=M (M=number of images)
# cov = A^T * A   M*M

cov_matrix2= np.cov(np.transpose(X_train_stacked))
cov_matrix2.shape

(966, 966)

In [9]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix2)

